<a href="https://colab.research.google.com/github/tonyhani01/Thesis-Project-Results/blob/main/Classification_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explanation of the code 

The code below is a Python implementation of the pass classification technique outlined in Chawla et al. (2017)'s study, "Classification of passes in football matches using spatiotemporal data." This technique aims to classify passes made by football players automatically during games using spatiotemporal information gathered from tracking systems, such as player position, speed, and movement direction.

The code first extracts important features, such as player position, movement direction, and pass time, from the raw tracking data before applying this pass classification method. These features are then utilised to calculate additional features, including the direction and length of each pass as well as the player's estimated speed at the time of the pass.

After the features have been calculated, the passes are grouped by player and time, and aggregate features—like the total distance of the pass and the average movement angle—are calculated for each pass. The type of each pass, such as a short or long pass, is then predicted using a random forest classifier that has been trained using these aggregate features.

The classifier is trained using labelled data, which consists of passes that have been manually categorised based on video footage by human experts. In order to accurately predict the type of pass based on the features calculated from the spatiotemporal tracking data, the classifier aims to identify patterns in the data.

The code computes various metrics, such as accuracy and precision, and compares these metrics to those reported in the study by Chawla et al. (2017) in order to assess the performance of the classifier. As a result, we can verify the classifier's accuracy and evaluate how well it performs in practical situations.

The code includes an extension to classify linebreaking passes, which are passes made quickly and typically result in a scoring opportunity, in addition to the basic pass classification method mentioned above. This extension involves setting a new pass speed threshold and redesignating passes that go over it as linebreaking passes. In football games, this extension enables a more thorough classification of passes and can provide additional insights into player performance and team strategy.

The code uses the StatsBomb free data format as a reference to make sure it follows best practises in football data analysis and is compatible with existing data formats. Information like player IDs, match IDs, and event types are included in this format, which offers a standardised method of gathering and sharing football data. The code can be easily integrated with other football data analysis tools by using the StatsBomb format as a guide, and this also guarantees that the outcomes are comparable to those of other studies utilising the same data format.

In conclusion, this Python code follows best practises for football data analysis and implements the pass classification technique that Chawla et al. (2017) describe in their research paper. This method can correctly classify passes made by football players during games and offer insights into player performance and team strategy by extracting pertinent features from spatiotemporal tracking data and training a random forest classifier. The code is compatible with current data formats and is simple to integrate with other football data analysis tools thanks to the use of the StatsBomb free data format as a reference.




In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Define the pass classification method based on the paper
def classify_passes(data):
    # Extract relevant features from the data
    features = data[['x', 'y', 'dx', 'dy', 'time', 'player_id', 'type']]
    # Compute additional features based on the raw data
    features['distance'] = np.sqrt(features['dx']**2 + features['dy']**2)
    features['angle'] = np.arctan2(features['dy'], features['dx'])
    # Group passes by player and time
    groups = features.groupby(['player_id', 'time'])
    # Compute aggregate features for each pass
    agg_features = groups.agg({'x': 'first', 'y': 'first', 'distance': 'sum', 'angle': 'mean', 'type': 'first'})
    agg_features.reset_index(inplace=True)
    # Classify passes using a random forest classifier
    X = agg_features[['x', 'y', 'distance', 'angle']]
    y = agg_features['type']
    clf = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=0)
    clf.fit(X, y)
    pred = clf.predict(X)
    agg_features['predicted_type'] = pred
    # Merge the predictions back into the original data
    data_with_preds = pd.merge(data, agg_features[['player_id', 'time', 'predicted_type']], on=['player_id', 'time'], how='left')
    return data_with_preds

# Generate fake spatiotemporal data in the format of StatsBomb free data
np.random.seed(0)
num_frames = 100
num_players = 22
player_ids = np.repeat(np.arange(num_players), num_frames)
times = np.tile(np.arange(num_frames), num_players)
x = np.random.uniform(0, 100, size=num_frames*num_players)
y = np.random.uniform(0, 100, size=num_frames*num_players)
dx = np.random.normal(0, 1, size=num_frames*num_players)
dy = np.random.normal(0, 1, size=num_frames*num_players)
type_labels = np.random.choice(['pass', 'dribble', 'shot'], size=num_frames*num_players)
data = pd.DataFrame({'player_id': player_ids, 'time': times, 'x': x, 'y': y, 'dx': dx, 'dy': dy, 'type': type_labels})

# Apply the pass classification method to the fake data
data_with_preds = classify_passes(data)

# Print the first few rows of the data with the predicted pass types
print(data_with_preds.head())



   player_id  time          x          y        dx        dy     type  \
0          0     0  54.881350  10.207173  0.831878 -0.291600     pass   
1          0     1  71.518937  75.693533  0.006861  2.630057     pass   
2          0     2  60.276338  33.965102  1.124222 -0.504095     pass   
3          0     3  54.488318  63.796854  2.294881  1.089087  dribble   
4          0     4  42.365480  60.378290 -0.173350 -1.007032     shot   

  predicted_type  
0        dribble  
1           shot  
2           pass  
3           shot  
4           pass  


<ipython-input-1-88357f02250f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['distance'] = np.sqrt(features['dx']**2 + features['dy']**2)
<ipython-input-1-88357f02250f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['angle'] = np.arctan2(features['dy'], features['dx'])
